In [1]:
import service.dtw as dtw
import utils.proxy_utils as pu
import core.yahoo_core as yh
import cufflinks as cf
from utils.cf_util import xaxes
import datetime
import pandas as pd

pu.proxy()
cf.set_config_file(offline=True, world_readable=True)

INFO:root:开启网络代理: socks5 localhost:10808


In [2]:
data_tem = "%Y-%m-%d"
code_ref_name = 'QQQ'
start = datetime.date(2000, 1, 1)
end = datetime.date(2023, 6, 20)
now = datetime.datetime.now().date()
code = yh.download_codes_batch_by_codes_str(codes_str=code_ref_name, start_str=end.strftime(data_tem))[0]
code_df_ref = code[['open', 'close', 'high', 'low']]
code_ref = code_df_ref.values
lang_code_df_ref = yh.download_codes_batch_by_codes_str(codes_str=code_ref_name, start_str='2023-01-01')[0][
    ['open', 'close', 'high', 'low']]

INFO:root:download_codes_batch_by_codes_str called. args: (), kwargs: {'codes_str': 'QQQ', 'start_str': '2023-06-20'}


[*********************100%***********************]  1 of 1 completed

INFO:root:download_codes_batch_by_codes_str called. args: (), kwargs: {'codes_str': 'QQQ', 'start_str': '2023-01-01'}



[*********************100%***********************]  1 of 1 completed


In [3]:
min_dist = dtw.find_pool(code_ref, pool='US_INDUSTRY', start_str=start.strftime(data_tem),
                         end_str=end.strftime(data_tem))

INFO:root:download_codes_batch_by_codes_str called. args: (), kwargs: {'codes_str': 'SPY,XLI,XLE,XLY,XLP,XLF,XLV,XLC,XLB,XLRE,XLK,XLU', 'start_str': '2000-01-01', 'end_str': '2023-06-20'}


[*********************100%***********************]  12 of 12 completed
code: XLC
code: XLRE
code: XLV
code: XLI
code: XLE
code: XLK
code: XLB
code: XLY
code: XLF
code: XLU
code: SPY
code: XLP


In [45]:
min_source = min_dist[89]
source_name = min_source['code']
source_start_str = min_source['start'].strftime(data_tem)
source_end_str = min_source['end'].strftime(data_tem)
source = \
yh.download_codes_batch_by_codes_str(codes_str=source_name, start_str=source_start_str, end_str=source_end_str)[0]
source_df = source[['open', 'close', 'high', 'low']]
source = source_df.values
source_df = source_df.reset_index(drop=False)
source_start = source_df.iloc[0]['Date'].to_pydatetime().date()
source_end = source_df.iloc[-1]['Date'].to_pydatetime().date()
source_df = source_df.set_index('Date', drop=False)

qf_source_ref = cf.QuantFig(source_df, title=source_name, legend='top', name=source_name)
fig_source = qf_source_ref.figure(up_color='green', down_color='red')
fig_source.update_xaxes(
    rangebreaks=[
        dict(values=xaxes(source_start, source_end, source_df))  # 剔除所有非交易日
    ]
)
fig_source.show()

fu_len = 20
fu_df = \
yh.download_codes_batch_by_codes_str(codes_str=source_name, start_str=source_start_str, end_str=now.strftime(data_tem))[
    0]
fu_df = fu_df.iloc[len(source_df):len(source_df) + fu_len][['open', 'close', 'high', 'low']]

INFO:root:download_codes_batch_by_codes_str called. args: (), kwargs: {'codes_str': 'XLF', 'start_str': '2019-03-28', 'end_str': '2019-05-01'}


[*********************100%***********************]  1 of 1 completed


INFO:root:download_codes_batch_by_codes_str called. args: (), kwargs: {'codes_str': 'XLF', 'start_str': '2019-03-28', 'end_str': '2023-07-23'}


[*********************100%***********************]  1 of 1 completed


In [5]:
qf_code_ref = cf.QuantFig(code_df_ref, title=code_ref_name, legend='top', name=code_ref_name)
fig = qf_code_ref.figure(up_color='green', down_color='red')
fig.update_xaxes(
    rangebreaks=[
        dict(values=xaxes(start, now, code_df_ref))  # 剔除所有非交易日
    ]
)
fig.show()

In [46]:
open = fu_df.iloc[0]['open']
close = code_df_ref.iloc[-1]['close']
xi = close / open
fu_df = fu_df * xi
fu_df = fu_df.reset_index(drop=False)
today = pd.to_datetime('today').normalize()
fu_df['Date'] = today + (fu_df['Date'] - fu_df['Date'].min())
end_date = fu_df.iloc[-1]['Date'].to_pydatetime().date()
fu_df = fu_df.set_index('Date', drop=True)

In [47]:
pdf = pd.concat([lang_code_df_ref, fu_df], axis=0)
qf_pdf = cf.QuantFig(pdf, title=code_ref_name + '_Predict', legend='top', name=code_ref_name + '_Predict')
qf_pdf.figure(up_color='green', down_color='red')
fig_pdf = qf_pdf.figure(up_color='green', down_color='red')
fig_pdf.update_xaxes(
    rangebreaks=[
        dict(values=xaxes(start, end_date, pdf))  # 剔除所有非交易日
    ]
)
fig_pdf.add_shape(type='line',
              x0='2023-07-21', y0=280,
              x1='2023-07-21', y1=370,
              line=dict(color='gray', dash='dash'))
fig_pdf.show()